In [1]:
import os
from os.path import join
from io import StringIO
from glob import glob
from textwrap import dedent

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

import QDYN
from src.single_sided_network_v1 import network_slh
from src.transfer_model_v1 import write_transfer_model

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [2]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2017-10-22_mg_method_comparison_transfer2.json',
}
register_doit_as_IPython_magic()

In [3]:
clusterjob.JobScript.read_defaults('./config/mlhpc_cluster.ini')

In [4]:
! qdyn_version

QDYN 2.0dev revision 36f75756deefe59f55ee5dec363429c8cf4f994f (master)
  features: no-check-cheby, no-check-newton, no-parallel-ham, use-mkl=sequential, use-mpi=intel, parallel-oct, backtraces, no-debug, no-no-ipo
  compiled with ifort on Tue Oct 10 08:56:03 2017 on host mlhpc2


$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# Compare variants of Krotov's method for optimization of a two-node state transfer

This compares to the earlier iteration of the same notebook in the following
ways:
* The independent-trajectory optimization is done with the state-to-state
functional, not the square-modulus functional. The earlier calculation was not
using the formula that is in the paper!
* Added cross-trajectory optimization
* Do not use MPI -- this presumes that the notebook is run on mlhpc (where we
have >64 cores)

In [5]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    root = './data/method_comparison_transfer2/'
    if frac_part == '00':
        T_str = "%s" % int_part
    else:
        T_str = '%s_%s' % (int_part, frac_part)
    rf = "T%s_%s_ntrajs%s" % (T_str, row['variant'], row['n_trajs'])
    return join(root, rf)

## action wrappers

In [6]:
from src.doit_actions_v2 import (
    run_traj_prop, update_config, wait_for_clusterjob,
    write_rho_prop_custom_config, collect_rho_prop_errors)

In [7]:
def submit_optimization(rf, n_trajs, variant, task):
    """Asynchronously run optimization, using either the
    standard QDYN implementation (variant = 'normal') or the 'crossoct' QDYN
    implementation at a particular revision (variant='cross')

    Does not support use of MPI, so this should only run on a workstation with
    more cores than the total number of trajectories
    """
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    bodies = {
        'normal': dedent(r'''
    {module_load}
    cd $CLUSTERJOB_WORKDIR
    pwd
    OMP_NUM_THREADS={n_trajs} qdyn_optimize --n-trajs={n_trajs} --J_T=J_T_ss .
    '''),
        'cross': dedent(r'''
    {module_load}
    module load prefix/crossoct

    qdyn_version | grep 66f6d24d2b07b9cc || {cancel_cmd}

    cd $CLUSTERJOB_WORKDIR
    pwd
    OMP_NUM_THREADS={n_trajs} qdyn_optimize --n-trajs={n_trajs} --J_T=J_T_cross .
    ''')}
    body = bodies[variant]
    taskname = "oct_%s" % task.name.split(":")[-1]
    jobscript = clusterjob.JobScript(
        body=body, jobname=taskname, workdir=rf,
        stdout='oct.log')
    jobscript.n_trajs = str(int(n_trajs))
    jobscript.resources['ppn'] = 1 # not using MPI
    jobscript.resources['threads'] = int(n_trajs)
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))


## custom uptodate routines

In [8]:
from src.qdyn_model_v2 import pulses_uptodate

## OCT task definitions

In [9]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    slh_models = {}
    n_nodes = 2
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        if rf in jobs:
            continue
        mcwf = {  #   variant => whether to use MCWF
            'rho': False, 'nonherm': False, 'independent': True,
            'cross': True}
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                # write the density matrix propagation data
                (write_transfer_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2,
                    mcwf=False, non_herm=False, # this defines rho-prop
                    config='config_rho')),
                # write the data for the optimization
                (write_transfer_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=mcwf[row['variant']],
                    non_herm=(row['variant'] != 'rho'),
                    lambda_a=row['lambda_a'], keep_pulses='all',
                    iter_stop=int(row['iter_stop']), J_T_conv=row['J_T_conv'],
                    config='config'))],
            'targets': [join(rf, 'config_rho'), join(rf, 'config')],
            'uptodate': [True, ] # up to date if targets exist
        }
    for job in jobs.values():
        yield job

In [10]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row
    (adjusting lambda_a, iter_stop and J_T_conv only)"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop']),
                    J_T_conv=row['J_T_conv']))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [11]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    optimization_variant = {
        'rho': 'normal',
        'nonherm': 'normal',
        'independent': 'normal',
        'cross': 'cross',
    }
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ],
                 dict(n_trajs=row['n_trajs'],
                      variant=optimization_variant[row['variant']]))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [12]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

Additional tasks for evaluating the optimization success through a propagation
in Liouville space are defined farther below

##  OCT Submission

In [13]:
params_data_str = r'''
# T  lambda_a      variant  n_trajs   iter_stop   J_T_conv
  5     0.001          rho        1        5000       1e-4
  5     0.001      nonherm        1        5000       1e-8
  5     0.001  independent        1        5000       1e-8
  5     0.001  independent        2        5000       1e-8
  5     0.001  independent        4        5000       1e-8
  5     0.001  independent        8        5000       1e-8
  5     0.001  independent       16        5000       1e-8
  5     0.001  independent       32        5000       1e-8
  5     0.001  independent       64        5000       1e-8
  5     0.001  cross              2        5000       1e-8
  5     0.001  cross              4        5000       1e-8
  5     0.001  cross              8        5000       1e-8
  5     0.001  cross             16        5000       1e-8
  5     0.001  cross             32        5000       1e-8
  5     0.001  cross             64        5000       1e-8
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['T', 'lambda_a', 'variant', 'n_trajs', 'iter_stop', 'J_T_conv'],
        converters={
            'T': float, 'lambda_a': float, 'variant': str,
            'n_trajs': int, 'iter_stop': int, 'J_T_conv': float})


In [14]:
print(params_df.to_string())

      T  lambda_a      variant  n_trajs  iter_stop      J_T_conv
0   5.0     0.001          rho        1       5000  1.000000e-04
1   5.0     0.001      nonherm        1       5000  1.000000e-08
2   5.0     0.001  independent        1       5000  1.000000e-08
3   5.0     0.001  independent        2       5000  1.000000e-08
4   5.0     0.001  independent        4       5000  1.000000e-08
5   5.0     0.001  independent        8       5000  1.000000e-08
6   5.0     0.001  independent       16       5000  1.000000e-08
7   5.0     0.001  independent       32       5000  1.000000e-08
8   5.0     0.001  independent       64       5000  1.000000e-08
9   5.0     0.001        cross        2       5000  1.000000e-08
10  5.0     0.001        cross        4       5000  1.000000e-08
11  5.0     0.001        cross        8       5000  1.000000e-08
12  5.0     0.001        cross       16       5000  1.000000e-08
13  5.0     0.001        cross       32       5000  1.000000e-08
14  5.0     0.001        

In [15]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(
    level=logging.DEBUG, filename='./data/method_comparison_transfer2.log')

In [16]:
%doit -n 20 wait_for_optimization

-- create_runfolder:./data/method_comparison_transfer2/T5_rho_ntrajs1
-- create_runfolder:./data/method_comparison_transfer2/T5_nonherm_ntrajs1
-- create_runfolder:./data/method_comparison_transfer2/T5_independent_ntrajs1
-- create_runfolder:./data/method_comparison_transfer2/T5_independent_ntrajs2
-- create_runfolder:./data/method_comparison_transfer2/T5_independent_ntrajs4
-- create_runfolder:./data/method_comparison_transfer2/T5_independent_ntrajs8
-- create_runfolder:./data/method_comparison_transfer2/T5_independent_ntrajs16
-- create_runfolder:./data/method_comparison_transfer2/T5_independent_ntrajs32
-- create_runfolder:./data/method_comparison_transfer2/T5_independent_ntrajs64
-- create_runfolder:./data/method_comparison_transfer2/T5_cross_ntrajs2
-- create_runfolder:./data/method_comparison_transfer2/T5_cross_ntrajs4
-- create_runfolder:./data/method_comparison_transfer2/T5_cross_ntrajs8
-- create_runfolder:./data/method_comparison_transfer2/T5_cross_ntrajs16
-- create_runfolde

## Evaluate error exactly from density matrix propagation

In [17]:
RHO_PROP_ITERS = np.arange(3000, step=10)

In [18]:
def task_rho_prop_create_custom_config():
    """For every optimized pulse, at intermediate iteration numbers,
    write a custom config file the will propagate this pulse"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        for oct_pulse_file in glob(join(rf, 'pulse1.oct.dat.0*')):
            oct_iter = int(os.path.splitext(oct_pulse_file)[-1][1:])
            if oct_iter not in RHO_PROP_ITERS:
                continue  # skip; we don't want to propagate *all* pulses
            assert os.path.isfile(oct_pulse_file.replace('pulse1', 'pulse2'))
            config_out = 'config_rho.%08d' % oct_iter
            yield {
                'name': str(rf) + "/%s" % config_out,
                'actions': [
                    (write_rho_prop_custom_config,
                     [rf, oct_iter, config_out])],
                'targets': [join(rf, config_out)],
                'uptodate': [True, ] # up to date if target exists
        }

In [19]:
def task_evaluate_rho_prop_error():
    """For every custom rho prop config file, perform the propagation"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        for custom_config in glob(join(rf, 'config_rho.0*')):
            oct_iter = int(os.path.splitext(custom_config)[-1][1:])
            yield {
                'name': str(rf) + "/%s" % custom_config,
                'actions': [
                    (run_traj_prop, [rf, ], dict(
                        n_trajs=1, n_procs=None, use_oct_pulses=False,
                        config=os.path.split(custom_config)[-1]))],
                'targets': [join(rf, 'P_target.dat.%08d' % oct_iter)],
                'file_dep': [custom_config],
                'uptodate': [True, ] # up to date if target exists
        }

In [20]:
def task_collect_rho_prop_errors():
    """For every custom rho prop config file, perform the propagation"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        P_target_obs_files = glob(join(rf, 'P_target.dat.0*'))
        target_file = join(rf, 'rho_prop_error.dat')
        yield {
            'name': target_file,
            'actions': [
                (collect_rho_prop_errors, P_target_obs_files,
                 dict(outfile=target_file))],
            'targets': [target_file],
            'file_dep': P_target_obs_files,
            'uptodate': [False, ]
            # We always create a new file
        }

In [21]:
%doit -n 20 rho_prop_create_custom_config

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000350
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00001270
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002880
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000420
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00001420
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002670
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002070
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002550
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002990
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rh

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00002910
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00001880
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000990
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00002320
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000750
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000560
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00002440
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00001360
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00001860
-

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00000400
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002220
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002460
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00000680
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002020
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002250
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002890
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002940
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002110
-

.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00000350
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001270
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00002880
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00000420
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001420
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00002670
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00002070
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00002550
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.0

.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001120
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001960
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001700
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00002530
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001350
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00000230
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001150
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00002310
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.0

...

.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00000080
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00000790
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00002210
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00002310
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00000650
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00001750
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00002400
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00002790
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00000630
.  rho_prop_create_custom_config:./data/method

.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00002840
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00000000
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00002590
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00002970
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00001720
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00001370
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00001840
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00000610
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs32/config_rho.00002010
.  rho_prop_create_custom_config:./data/method

.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001280
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000630
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000080
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001180
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002400
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002120
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002260
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001440
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000480
.  rho_prop_create_custom_config:./data/method

.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000390
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002750
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002850
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000030
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000550
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001650
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002820
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001220
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002030
.  rho_prop_create_custom_config:./data/method

.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002640
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002470
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002450
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001520
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000500
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002870
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001880
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002910
.  rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002320
.  rho_prop_create_custom_config:./data/method

In [22]:
%doit -n 20 evaluate_rho_prop_error

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002800
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000540
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000990
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000630
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000830
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002350
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002600
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002780
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000240
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rh

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000270
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00002470
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000250
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00002130
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00002880
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000900
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000130
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000390
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00002660
-

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00000940
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00001750
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002610
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002710
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00001980
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00000400
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00000420
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00000200
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00001620
-

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002760
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002900
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002730
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00000790
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00001070
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002850
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00000060
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002840
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.0

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002200
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00000040
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00000860
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00001630
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002390
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002290
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00000850
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002810
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00001400
-- rho_prop_create_custom_config:./data/method_comparis

...

.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000810
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002970
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000690
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001990
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000410
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002220
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./

.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000120
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001190
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001270
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002080
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000260
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002030
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./

.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002770
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002590
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002910
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002140
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001010
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002150
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./

.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000390
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002660
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002320
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001880
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001560
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002930
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./

.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001450
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00002020
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001730
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001120
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00001030
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./data/method_comparison_transfer2/T5_cross_ntrajs64/config_rho.00000910
.  evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs64/./

In [23]:
%doit -n 20 collect_rho_prop_errors

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00001810
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002300
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002570
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000160
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002700
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00001870
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00000610
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00001690
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rho_ntrajs1/config_rho.00002520
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_rh

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00001860
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00001920
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00001770
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00001320
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000590
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000100
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00001310
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000640
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs1/config_rho.00000550
-

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00001820
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00000830
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002510
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002680
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00002040
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00001520
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00000480
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00001120
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs8/config_rho.00001020
-

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002920
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00000630
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002370
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002200
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00001360
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00001480
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00001530
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.00002630
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_independent_ntrajs64/config_rho.0

-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00000010
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002760
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002340
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00000680
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002590
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002440
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00000540
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00001720
-- rho_prop_create_custom_config:./data/method_comparison_transfer2/T5_cross_ntrajs8/config_rho.00002490
-- rho_prop_create_custom_config:./data/method_comparis

...

-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs4/./data/method_comparison_transfer2/T5_independent_ntrajs4/config_rho.00002900
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs4/./data/method_comparison_transfer2/T5_independent_ntrajs4/config_rho.00000410
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs4/./data/method_comparison_transfer2/T5_independent_ntrajs4/config_rho.00000420
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs4/./data/method_comparison_transfer2/T5_independent_ntrajs4/config_rho.00000040
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs4/./data/method_comparison_transfer2/T5_independent_ntrajs4/config_rho.00000780
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs4/./data/method_comparison_transfer2/T5_independent_ntrajs4/config_rho.00001960
-- evaluate_rho_prop_e

-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs32/./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001010
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs32/./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001570
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs32/./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00002600
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs32/./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00001940
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs32/./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00000150
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_independent_ntrajs32/./data/method_comparison_transfer2/T5_independent_ntrajs32/config_rho.00002080
-- evaluat

-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs2/./data/method_comparison_transfer2/T5_cross_ntrajs2/config_rho.00001100
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs2/./data/method_comparison_transfer2/T5_cross_ntrajs2/config_rho.00002610
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs2/./data/method_comparison_transfer2/T5_cross_ntrajs2/config_rho.00001980
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs2/./data/method_comparison_transfer2/T5_cross_ntrajs2/config_rho.00002300
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs2/./data/method_comparison_transfer2/T5_cross_ntrajs2/config_rho.00002600
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs2/./data/method_comparison_transfer2/T5_cross_ntrajs2/config_rho.00002380
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs2/./data/method_c

-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs16/./data/method_comparison_transfer2/T5_cross_ntrajs16/config_rho.00002970
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs16/./data/method_comparison_transfer2/T5_cross_ntrajs16/config_rho.00000160
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs16/./data/method_comparison_transfer2/T5_cross_ntrajs16/config_rho.00002580
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs16/./data/method_comparison_transfer2/T5_cross_ntrajs16/config_rho.00000320
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs16/./data/method_comparison_transfer2/T5_cross_ntrajs16/config_rho.00002690
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs16/./data/method_comparison_transfer2/T5_cross_ntrajs16/config_rho.00001010
-- evaluate_rho_prop_error:./data/method_comparison_transfer2/T5_cross_ntrajs16/./

.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_rho_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_nonherm_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_independent_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_independent_ntrajs2/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_independent_ntrajs4/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_independent_ntrajs8/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_independent_ntrajs16/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_independent_ntrajs32/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_independent_ntrajs64/rho_prop_error.dat
.  collect_rho_prop_errors:./data/method_comparison_transfer2/T5_cross_nt